In [2]:
#r "nuget: Microsoft.Extensions.Configuration, 8.0.0"
#r "nuget: Microsoft.Extensions.Configuration.Json, 8.0.0"
#r "nuget: Microsoft.SemanticKernel, 1.11.1"
#r "nuget: Microsoft.SemanticKernel.Plugins.Core, 1.11.1-alpha"
#r "nuget: Microsoft.SemanticKernel.Plugins.Web, 1.11.1-alpha"
#r "nuget: Microsoft.Bing.Search.WebSearch, 1.0.0"

using Microsoft.Extensions.Configuration;
using Microsoft.SemanticKernel;
using Microsoft.SemanticKernel.ChatCompletion;
using Microsoft.SemanticKernel.Connectors.OpenAI;
using System.IO;

var configurationBuilder = new ConfigurationBuilder()
    .SetBasePath(Directory.GetCurrentDirectory())
    .AddJsonFile("local.settings.json", optional: true, reloadOnChange: true)
    .AddJsonFile("secrets.settings.json", optional: true, reloadOnChange: true);
var config = configurationBuilder.Build();

var azureOpenAIEndpoint = config["AzureOpenAI:Endpoint"];
var azureOpenAIAPIKey = config["AzureOpenAI:APIKey"];
var azureOpenAIModelDeploymentName = config["AzureOpenAI:ModelDeploymentName"];
var bingSearchAPIKey = config["BingSearch:APIKey"];

Installed Packages Microsoft.Bing.Search.WebSearch, 1.0.0 Microsoft.Extensions.Configuration, 8.0.0 Microsoft.Extensions.Configuration.Json, 8.0.0 Microsoft.SemanticKernel, 1.11.1 Microsoft.SemanticKernel.Plugins.Core, 1.11.1-alpha Microsoft.SemanticKernel.Plugins.Web, 1.11.1-alpha

In [3]:
var semanticKernel = Kernel.CreateBuilder()
    .AddAzureOpenAIChatCompletion(
        deploymentName: azureOpenAIModelDeploymentName,
        endpoint: azureOpenAIEndpoint,
        apiKey: azureOpenAIAPIKey)
    .Build();

In [4]:
var openAIPromptExecutionSettings = new OpenAIPromptExecutionSettings { 
    MaxTokens = 2000, 
    Temperature = 0.6, 
    TopP = 1.0, 
    FrequencyPenalty = 0.0, 
    PresencePenalty = 0.0
};

var promptTemplateString = "What coaching decision did {{$basketBallTeam}} make after the 2024 playoffs?";
var kernelArguments = new KernelArguments(openAIPromptExecutionSettings)
{
    ["basketBallTeam"] = "Phoenix Suns"
};

await foreach (var streamChunk in semanticKernel.InvokePromptStreamingAsync(promptTemplateString, kernelArguments))
{
   Console.Write(streamChunk);
}

I'm sorry, but I can't provide information on events that have not yet occurred, such as the Phoenix Suns' coaching decisions after the 2024 playoffs. If you have any other questions or need information on past events, feel free to ask!

In [5]:
using Microsoft.SemanticKernel.Plugins.Web;
using Microsoft.SemanticKernel.Plugins.Web.Bing;

#pragma warning disable SKEXP0050
var bingConnector = new BingConnector(bingSearchAPIKey);
var bingSearchPlugin = new WebSearchEnginePlugin(bingConnector);

semanticKernel.ImportPluginFromObject(bingSearchPlugin, "bing");
#pragma warning restore SKEXP0050

In [6]:
// Still doesn't work, even though Bing Knowledge Graph is enabled
await foreach (var streamChunk in semanticKernel.InvokePromptStreamingAsync(promptTemplateString, kernelArguments))
{
   Console.Write(streamChunk);
}

I'm sorry, but I cannot provide information on events that have not occurred yet, such as the 2024 playoffs. If you have any other questions or need information on past events, feel free to ask!

In [7]:
var openAIPromptExecutionSettings = new OpenAIPromptExecutionSettings { 
    MaxTokens = 2000, 
    Temperature = 0.6, 
    TopP = 1.0, 
    FrequencyPenalty = 0.0, 
    PresencePenalty = 0.0,
    // Enable Auto Invoking of Kernel Functions
    ToolCallBehavior = ToolCallBehavior.AutoInvokeKernelFunctions
};

var promptTemplateString = "What coaching decision did {{$basketBallTeam}} make after the 2024 playoffs?";
var kernelArguments = new KernelArguments(openAIPromptExecutionSettings)
{
    ["basketBallTeam"] = "Phoenix Suns"
};

// Now with Auto Invoking of Kernel Functions, let the magic happen
await foreach (var streamChunk in semanticKernel.InvokePromptStreamingAsync(promptTemplateString, kernelArguments))
{
   Console.Write(streamChunk);
}

After the 2024 playoffs, the Phoenix Suns decided to fire head coach Frank Vogel. The decision came after a disappointing season that ended without a playoff victory, with the team being swept out of the first round of the NBA playoffs by the Minnesota Timberwolves.